<a href="https://colab.research.google.com/github/danareyes19/Topicos-IA/blob/main/Recorrido%20por%20Colab/GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lectura del set de datos

In [ ]:
import numpy as np
import os
import gzip

def load_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas
    

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta= '/content/gdrive/MyDrive/Tópicos-IA/fashion_mnist_data'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

Mounted at /content/gdrive


# Reajustar el tamaño de los datos

In [ ]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

# Importar tensorflow 2 (ya incluye Keras)

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.8.0


## Creación del modelo

In [ ]:
tf.random.set_seed(200)
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5,5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5,5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 28, 28, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                        

# Verificar disponibilidad de la GPU

In [ ]:
nombre_gpu = tf.test.gpu_device_name()

if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


# Checkpoint

In [ ]:
checkpoint_path = '/content/gdrive/MyDrive/Tópicos-IA/GPU/modeloGPU'
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose =1)

# Entrenamiento con GPU

In [ ]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train, Y_train, validation_data=(X_test,Y_test), batch_size=128, epochs=2, verbose=1, callbacks = [cp_callback])
  
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

Epoch 1/2
468/468 [==============================] - ETA: 0s - loss: 0.3227 - accuracy: 0.8834
Epoch 1: saving model to /content/gdrive/MyDrive/Tópicos-IA/GPU/modeloGPU
468/468 [==============================] - 19s 41ms/step - loss: 0.3227 - accuracy: 0.8834 - val_loss: 0.2732 - val_accuracy: 0.9019
Epoch 2/2
467/468 [============================>.] - ETA: 0s - loss: 0.2860 - accuracy: 0.8952
Epoch 2: saving model to /content/gdrive/MyDrive/Tópicos-IA/GPU/modeloGPU
468/468 [==============================] - 19s 40ms/step - loss: 0.2857 - accuracy: 0.8953 - val_loss: 0.2979 - val_accuracy: 0.8954


In [ ]:
print('Tiempo de entrenamiento: ' + str(gpu_time) + ' segundos')

Tiempo de entrenamiento: 41.03740647700005 segundos
